In [148]:
# Initialize Otter
import otter
grader = otter.Notebook("cs109a_hw7.ipynb")

In [1]:
# RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/"
    "themes/static/css/cs109.css"
).text
HTML(styles)

### Import Libraries

In [4]:
import random
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [5]:
# Select URLs for scraping
url_10 = "https://en.wikipedia.org/wiki/2010_FIFA_World_Cup_squads"
url_14 = "https://en.wikipedia.org/wiki/2014_FIFA_World_Cup_squads"
url_18 = "https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads"
url_22 = "https://en.wikipedia.org/wiki/2022_FIFA_World_Cup_squads"

page_10 = requests.get(url_10)
sleep(2)

page_14 = requests.get(url_14)
sleep(2)

page_18 = requests.get(url_18)
sleep(2)

page_22 = requests.get(url_22)
sleep(2)

page_10.status_code, page_14.status_code, page_18.status_code, page_22.status_code

(200, 200, 200, 200)

In [6]:
# Extract page HTML
raw_html_10 = page_10.text
raw_html_14 = page_14.text
raw_html_18 = page_18.text
raw_html_22 = page_22.text

page_soup_10 = BeautifulSoup(raw_html_10, 'html.parser')
page_soup_14 = BeautifulSoup(raw_html_14, 'html.parser')
page_soup_18 = BeautifulSoup(raw_html_18, 'html.parser')
page_soup_22 = BeautifulSoup(raw_html_22, 'html.parser')

In [7]:
# Define function to extract the list of countries
# participating in each world cup
def parse_country_list(page_soup):
    
    country_list = []
    country_elts = page_soup.select('h3')
    for tag in country_elts:
        modified_tag = tag.find('span', attrs = {'class': 'mw-headline'})
        if modified_tag is not None:  
            country_list.append(modified_tag.text)
            
    return country_list    

# Extract the participating country names for each tournament
countries_10 = parse_country_list(page_soup_10)
countries_10 = countries_10[:-5]

countries_14 = parse_country_list(page_soup_14)
countries_14 = countries_14[:-4]

countries_18 = parse_country_list(page_soup_18)
countries_18 = countries_18[:-5]

countries_22 = parse_country_list(page_soup_22)
countries_22 = countries_22[:-6]

print(countries_10)
print(countries_14)
print(countries_18)
print(countries_22)

['France', 'Mexico', 'South Africa', 'Uruguay', 'Argentina', 'Greece', 'Nigeria', 'South Korea', 'Algeria', 'England', 'Slovenia', 'United States', 'Australia', 'Germany', 'Ghana', 'Serbia', 'Cameroon', 'Denmark', 'Japan', 'Netherlands', 'Italy', 'New Zealand', 'Paraguay', 'Slovakia', 'Brazil', 'Ivory Coast', 'North Korea', 'Portugal', 'Chile', 'Honduras', 'Spain', 'Switzerland']
['Brazil', 'Cameroon', 'Croatia', 'Mexico', 'Australia', 'Chile', 'Netherlands', 'Spain', 'Colombia', 'Greece', 'Ivory Coast', 'Japan', 'Costa Rica', 'England', 'Italy', 'Uruguay', 'Ecuador', 'France', 'Honduras', 'Switzerland', 'Argentina', 'Bosnia and Herzegovina', 'Iran', 'Nigeria', 'Germany', 'Ghana', 'Portugal', 'United States', 'Algeria', 'Belgium', 'Russia', 'South Korea']
['Egypt', 'Russia', 'Saudi Arabia', 'Uruguay', 'Iran', 'Morocco', 'Portugal', 'Spain', 'Australia', 'Denmark', 'France', 'Peru', 'Argentina', 'Croatia', 'Iceland', 'Nigeria', 'Brazil', 'Costa Rica', 'Serbia', 'Switzerland', 'Germany',

In [8]:
import difflib

# Import player statisctics data and remove duplicate names
fifa_cleaned = pd.read_csv("data/fifa_cleaned.csv")
fifa_cleaned = fifa_cleaned.drop_duplicates(subset = 'full_name', keep = "first")

# Keep only the list of full names
fifa_names = list(fifa_cleaned.full_name)

# Create function which find squad and squad statistic data
def squad_parser(page_soup, countries_list):
    
    tables = page_soup.select('table.sortable.wikitable.plainrowheaders')
    players_list = []
    
    # Find the list of players for each country
    for i in range(len(countries_list)):
        rows = tables[i].select('.nat-fs-player')
        team_players = []
        
        for row in rows:
            
            player_dict = dict()
            
            # Scrape player position
            position = row.select_one('td a').text
            
            # Scrape player page for full name
            player_page = row.select_one('th a').get('href')
            player_page = "https://en.wikipedia.org" + player_page
            player_html = requests.get(player_page).text
            player_html = BeautifulSoup(player_html, 'html.parser')
            
            name = player_html.select_one('p b')
            full_name = name.text.strip()
            name_as_list = full_name.split()
            
            # Search for most similar name in FIFA video game name list
            if len(name_as_list) >= 2:
                short_name = name_as_list[0] + " " + name_as_list[-1]
                similar_name = difflib.get_close_matches(short_name, fifa_names, cutoff = 0.75)
                if len(similar_name) == 0:
                    similar_name = difflib.get_close_matches(full_name, fifa_names)
            else:
                similar_name = difflib.get_close_matches(full_name, fifa_names)
                
            if len(similar_name) >= 1:
                similar_name = similar_name[0]
            else:
                similar_name = None
            
            # Save player statistics
            player_dict['name'] = full_name
            player_dict['position'] = position
            player_dict['country'] = countries_list[i]
            
            # Find player ability data, if it exists
            if similar_name is None:
                player_dict['overall_rating'] = np.nan
                player_dict['dribbling'] = np.nan
                player_dict['freekick_accuracy'] = np.nan
                player_dict['agility'] = np.nan
                player_dict['ball_control'] = np.nan
                player_dict['interceptions'] = np.nan
                player_dict['penalties'] = np.nan
            
            else:
                fifa_stats = fifa_cleaned.loc[fifa_cleaned['full_name'] == similar_name]
                player_dict['overall_rating'] = int(fifa_stats.overall_rating)
                player_dict['dribbling'] = int(fifa_stats.dribbling)
                player_dict['freekick_accuracy'] = int(fifa_stats.freekick_accuracy)
                player_dict['agility'] = int(fifa_stats.agility)
                player_dict['ball_control'] = int(fifa_stats.ball_control)
                player_dict['interceptions'] = int(fifa_stats.interceptions)
                player_dict['penalties'] = int(fifa_stats.penalties)
            
            team_players.append(player_dict)
        
        # Mean imputation
        team_df = pd.DataFrame(team_players)
        team_df = team_df.dropna()
        
        # Find team means
        overall_rating = np.mean(team_df.overall_rating)
        dribbling = np.mean(team_df.dribbling)
        freekick_accuracy = np.mean(team_df.freekick_accuracy)
        agility = np.mean(team_df.agility)
        ball_control = np.mean(team_df.ball_control)
        interceptions = np.mean(team_df.interceptions)
        penalties = np.mean(team_df.penalties)
        
        # Assign means for missing data entries
        for player_dict in team_players:
            if np.isnan(player_dict['overall_rating']):
                player_dict['overall_rating'] = overall_rating
                player_dict['dribbling'] = dribbling
                player_dict['freekick_accuracy'] = freekick_accuracy
                player_dict['agility'] = agility
                player_dict['ball_control'] = ball_control
                player_dict['interceptions'] = interceptions
                player_dict['penalties'] = penalties
            players_list.append(player_dict)
    return players_list

# Find players & player statistics for each world cup
players_10 = squad_parser(page_soup_10, countries_10)
players_14 = squad_parser(page_soup_14, countries_14)
players_18 = squad_parser(page_soup_18, countries_18)
players_22 = squad_parser(page_soup_22, countries_22)

# Save world cup data
with open("data/players_10.json", "w") as f:
    json.dump(players_10, f)
    
with open("data/players_14.json", "w") as f:
    json.dump(players_14, f)
    
with open("data/players_18.json", "w") as f:
    json.dump(players_18, f)
    
with open("data/players_22.json", "w") as f:
    json.dump(players_22, f)

In [9]:
# Reformat data into dataframes
players_10_df = pd.DataFrame(players_10)
players_14_df = pd.DataFrame(players_14)
players_18_df = pd.DataFrame(players_18)
players_22_df = pd.DataFrame(players_22)

# Save data as CSVs
players_10_df.to_csv('data/players_2010_world_cup.csv')
players_14_df.to_csv('data/players_2014_world_cup.csv')
players_18_df.to_csv('data/players_2018_world_cup.csv')
players_22_df.to_csv('data/players_2022_world_cup.csv')

players_22_df.head(10)

,name,position,country,overall_rating,dribbling,freekick_accuracy,agility,ball_control,interceptions,penalties
0,Hernán Ismael Galíndez,GK,Ecuador,53.0,62.0,33.0,56.0,56.0,41.0,50.0
1,Felix Eduardo Torres Caicedo,DF,Ecuador,71.0,68.0,38.0,72.0,71.0,69.0,45.0
2,Piero Martín Hincapié Reyna,DF,Ecuador,64.0,13.0,13.0,35.0,13.0,29.0,20.0
3,Robert Abel Arboleda Escobar,DF,Ecuador,66.0,68.0,45.0,59.0,71.0,37.0,44.0
4,José Adoni Cifuentes Charcopa,MF,Ecuador,72.0,74.0,77.0,74.0,75.0,26.0,72.0
5,William Joel Pacho Tenorio,DF,Ecuador,58.0,57.0,28.0,54.0,64.0,17.0,53.0
6,Pervis Josué Estupiñán Tenorio,DF,Ecuador,66.0,49.0,36.0,63.0,61.0,58.0,40.0
7,Carlos Armando Gruezo Arboleda,MF,Ecuador,67.0,32.0,26.0,47.0,41.0,73.0,40.0
8,Eduar Ayrton Preciado García,MF,Ecuador,71.0,73.0,68.0,79.0,68.0,37.0,71.0
9,Romario Andrés Ibarra Mina,MF,Ecuador,71.0,57.0,58.0,74.0,64.0,68.0,55.0
